In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
torch.cuda.is_available()
import torchvision
import torchvision.datasets as datasets
import time
import torch.jit as jit
from torch import nn
from torch import optim
import os
from scipy import io
import pickle
device='cpu'
torch.set_default_tensor_type(torch.FloatTensor)

from matplotlib import cm

from Util_Benchmarking import *


c:\Users\kilia\anaconda3\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ..\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [5]:
T_data=1000
Data=Data_Manager("MACKEY_manuscript",T_data)

S_first=Data.S
Y_first=Data.Y

Data.Init_ESN(N=500, N_in=1, N_av=10, alpha=0.9, rho=0.99, gamma=0.15)

Data.ESN_Process()

X_esn=Data.X


Data.ESN=False
T_seg=10

Data.Segmentation(T_seg)

S_seg=Data.S
Y_seg=Data.Y

val_prc=0.1
Data.Splitting(val_prc)

S_tr=Data.S_tr
S_val=Data.S_val
S_te=Data.S_te

Y_tr=Data.Y_tr
Y_val=Data.Y_val
Y_te=Data.Y_te


fig, axs = plt.subplots(2, 2,figsize=(20,10))

N_y=3
T_plot=100

axs[0,0].plot(S_first[0:T_plot,:].to('cpu'),'black')

N_y=3
for n in range(1,N_y):
    axs[0,0].plot(Y_first[0:T_plot,n].to('cpu'),'red')
    
axs[0,0].set(xlabel='Step number',ylabel='Signal and Targets')


axs[0,1].plot(S_first[0:T_plot,:].to('cpu'), color='black', linestyle='dashed')

N_esn=100
cl=cm.inferno(np.linspace(0,1,N_esn))

for n in range(1,N_esn):
    axs[0,1].plot(X_esn[0:T_plot,n].to('cpu'),color=cl[n,:])
    
axs[0,1].set(xlabel='Step number',ylabel='Signal and ESN response')

for n in range(T_seg):
    axs[1,0].plot(S_seg[0:T_plot,n].to('cpu'),'black')

N_y=2
for n in range(1,N_y):
    axs[1,0].plot(Y_seg[0:T_plot,n].to('cpu'),'red')
    
axs[1,0].set(xlabel='Step number',ylabel='Signal and Targets')



ts_tr=torch.arange(0,S_tr.size()[0])
ts_val=torch.arange(S_tr.size()[0],S_tr.size()[0]+S_val.size()[0])
ts_te=torch.arange(S_tr.size()[0]+S_val.size()[0],S_tr.size()[0]+S_val.size()[0]+S_te.size()[0])

T_tr_plot=100
axs[1,1].plot(ts_tr[-T_tr_plot:].to('cpu'),S_tr.to('cpu')[-T_tr_plot:,-1],color='black')
axs[1,1].plot(ts_val.to('cpu'),S_val.to('cpu')[:,-1],'black',linestyle='dashed')
axs[1,1].plot(ts_te.to('cpu'),S_te.to('cpu')[:,-1],'black',linestyle='dashdot')

axs[1,1].plot(ts_tr[-T_tr_plot:].to('cpu'),Y_tr.to('cpu')[-T_tr_plot:,2],color='red')
axs[1,1].plot(ts_val.to('cpu'),Y_val.to('cpu')[:,2],linestyle='dashed',color='red')
axs[1,1].plot(ts_te.to('cpu'),Y_te.to('cpu')[:,2],linestyle='dashdot',color='red')

axs[1,1].set(xlabel='Step number',ylabel='Splitting')



: 

In [ ]:
T_data=10000
Data=Data_Manager("MACKEY",T_data)


Model_Id='MLP'
val_prc=0.1

if Model_Id=='ESN':
    
    N_ESN=500
    N_in=1
    N_av=10
    alpha=0.9
    rho=0.99
    gamma=0.15
    
    Data.Init_ESN(N_ESN, N_in, N_av, alpha, rho, gamma)
    Data.ESN_Process()
    Data.Splitting(val_prc)
    
    F_Ns=torch.tensor([N_ESN,Data.Y_tr.size()[1]])
    
    eta=0.0001
    weight_decay=0
    model=Model_Learning(F_Ns, eta, weight_decay)
    
    T_seq=1
    
if Model_Id=='MLP':
    
    T_seg=5
    Data.Segmentation(T_seg)
    Data.Splitting(val_prc)

    F_Ns=torch.tensor([Data.S_tr.size()[1],200,200,Y_tr.size()[1]])
    
    eta=0.001
    weight_decay=0
    model=Model_Learning(F_Ns, eta, weight_decay)
    
    T_seq=1
    
if Model_Id=='N_ODE':
    
    N_ODE=True
    
    T_seg=5
    Data.Segmentation(T_seg)
    Data.Splitting(val_prc)

    
    F_Ns=torch.tensor([Data.S_tr.size()[1],200,200,Data.S_tr.size()[1]])
    
    eta=0.001
    weight_decay=0
    model=Model_Learning(F_Ns, eta, weight_decay, N_ODE)
    
    T_seq=20
    
batch_size=20


In [ ]:
N_train=20000
N_checkpoint=500

N_check=int(np.ceil(N_train/N_checkpoint))
ind_help=0

N_chunks=1

MSE_Train=torch.zeros([N_train])

MSE_Tr=torch.zeros([N_check])
MSE_Val=torch.zeros([N_check])
MSE_Te=torch.zeros([N_check])

t0s=torch.tensor(0.)


for k in range(N_train):
    
    X_b, Y_b=Data.Batch(batch_size, T_seq)
    
    if model.N_ODE:

        var_help=X_b.clone()
        X_b=var_help[:,:,0:-1].clone()
        Y_b=var_help[:,:,1:]
    
    err,_=model.forward(X_b,Y_b,t0s)
    MSE_Train[k]=err.detach()
    
    if k%N_checkpoint==0: 
        
        
        if k>0:
            
            mse_mean=torch.mean(MSE_Train[k-N_checkpoint:k])
        
            MSE_Tr[ind_help]=mse_mean
            
        else:
            
            MSE_Tr[ind_help]=MSE_Train[k]
            
        
        ## VALIDATING ##
        #############
                            
        X_b,Y_b=Data.Testing()
        
        if model.N_ODE:
            
            var_help=X_b.clone()
            X_b=var_help[:,:,0:-1].clone()
            Y_b=var_help[:,:,1:]
        
        err_, y_=model.Seqforward(X_b,Y_b,t0s)
        
        MSE_Val[ind_help]=err_.detach()
        
        ## TESTING ##
        #############
        
        X_b,Y_b=Data.Testing(False)
        
        if model.N_ODE:
        
            var_help=X_b.clone()
            X_b=var_help[:,:,0:-1].clone()
            Y_b=var_help[:,:,1:]
        
        err_, y_=model.Seqforward(X_b,Y_b,t0s)
        
        MSE_Te[ind_help]=err_.detach()
        
        print('Errors: ', MSE_Tr[ind_help], MSE_Val[ind_help] , MSE_Te[ind_help])
        
        ind_help=ind_help+1

Run the MLP comparison hyperparameters used in the manuscript

In [3]:


T_datax=[50,100,150,200,250,500,1000,2000]
T_segx=[0,1,2,3,5,10,20]
## Then number of nodes in the hidden layers, something like
N_hiddenx=[5,10,20,40,60,80,100,150,200]  
N_train=20000
N_checkpoint=19999
params = []
for T_data in T_datax:
    for T_seg in T_segx:
        for N_hidden in N_hiddenx:
            params.append([T_data,T_seg,N_hidden])
print(len(params))
import sys

val_prc=0.1
for i in range(len(params)):
    param = params[i]
    T_data=param[0]
    T_seg = param[1]
    N_hidden=param[2]

    Data=Data_Manager("MACKEY_manuscript",T_data)
    

    print(T_data,T_seg,N_hidden)

    if T_seg>0:
        Data.Segmentation(T_seg)

    Data.Splitting(val_prc)
    

    F_Ns=torch.tensor([Data.S_tr.size()[1],N_hidden,N_hidden,Data.Y_tr.size()[1]])  ## Definition of the MLP shape

    eta=0.001 ## Learning rate
    weight_decay=0 ## Learning rate of the L2 penalty term 
    model=Model_Learning(F_Ns, eta, weight_decay)

    T_seq=1
    
    
    N_check=int(np.ceil(N_train/N_checkpoint))
    ind_help=0

    N_chunks=1

    MSE_Train=torch.zeros([N_train])

    MSE_Tr=torch.zeros([N_check])
    MSE_Val=torch.zeros([N_check])
    MSE_Te=torch.zeros([N_check])

    t0s=torch.tensor(0.)

    batch_size = 20
    for k in range(N_train):

        X_b, Y_b=Data.Batch(batch_size, T_seq)

        if model.N_ODE:

            var_help=X_b.clone()
            X_b=var_help[:,:,0:-1].clone()
            X_b=torch.concat([X_b,X_b],1)

            Y_b=var_help[:,:,1:]

        err,_=model.forward(X_b,Y_b,t0s)
        MSE_Train[k]=err.detach()

        if k%N_checkpoint==0: 


            if k>0:

                mse_mean=torch.mean(MSE_Train[k-N_checkpoint:k])

                MSE_Tr[ind_help]=mse_mean

            else:

                MSE_Tr[ind_help]=MSE_Train[k]


            ## VALIDATING ##
            #############

            X_val,Y_val=Data.Testing()

            if model.N_ODE:

                var_help=X_val.clone()
                X_val=var_help[:,:,0:-1].clone()
                X_val=torch.concat([X_val,X_val],1)

                Y_val=var_help[:,:,1:]

            err_, y_val=model.Seqforward(X_val,Y_val,t0s)

            MSE_Val[ind_help]=err_.detach()

            ## TESTING ##
            #############

            X_te,Y_te=Data.Testing(False)

            if model.N_ODE:

                var_help=X_te.clone()
                X_te=var_help[:,:,0:-1].clone()
                X_te=torch.concat([X_te,X_te],1)

                Y_te=var_help[:,:,1:]

            err_, y_te=model.Seqforward(X_te,Y_te,t0s)

            MSE_Te[ind_help]=err_.detach()

            Nx_plot=3
            Ny_plot=3

            fig, axs = plt.subplots(Nx_plot, Ny_plot,figsize=(20,10))

            print('Iteration number: ', k)
            for l in range(Nx_plot):

                for m in range(Ny_plot):

                    t_pred=l*Nx_plot+m
                    axs[l,m].plot(y_val.detach().to('cpu')[0,t_pred,:],'black')
                    axs[l,m].plot(Y_val.detach().to('cpu')[0,t_pred,:],'red')

                    axs[l,m].legend(['Prediction','Target'])
                    #tit=
                    axs[l,m].title.set_text('Delay: '+str(t_pred))


            plt.savefig(os.path.join('MLP','Plots_T_data_{}_T_seg_{}_N_hidden_{}.png'.format(T_data,T_seg,N_hidden)))
            #plt.show()
            print('Errors: ', MSE_Tr[ind_help], MSE_Val[ind_help] , MSE_Te[ind_help])

            ind_help=ind_help+1
    #print(y_val)
    print(y_te)
    print(Y_te)
    
    torch.save(Y_te,os.path.join('MLP','xY_te_T_data_{}_T_seg_{}_N_hidden_{}.pt'.format(T_data,T_seg,N_hidden)))
    torch.save(y_te,os.path.join('MLP','y_te_T_data_{}_T_seg_{}_N_hidden_{}.pt'.format(T_data,T_seg,N_hidden)))
    torch.save(Y_val,os.path.join('MLP','xY_val_T_data_{}_T_seg_{}_N_hidden_{}.pt'.format(T_data,T_seg,N_hidden)))
    torch.save(y_val,os.path.join('MLP','y_val_T_data_{}_T_seg_{}_N_hidden_{}.pt'.format(T_data,T_seg,N_hidden)))
    
    torch.save(MSE_Tr,os.path.join('MLP','MSE_Tr_T_data_{}_T_seg_{}_N_hidden_{}.pt'.format(T_data,T_seg,N_hidden)))
    torch.save(MSE_Val,os.path.join('MLP','MSE_Val_T_data_{}_T_seg_{}_N_hidden_{}.pt'.format(T_data,T_seg,N_hidden)))
    torch.save(MSE_Te,os.path.join('MLP','MSE_Te_T_data_{}_T_seg_{}_N_hidden_{}.pt'.format(T_data,T_seg,N_hidden)))

504
50 0 5


NameError: name 'batch_size' is not defined

Run the ESN hyperparameters from the manuscript

In [5]:
T_datax=[50,100,150,200,250,500,1000,2000]
N_ESNx=np.int32(np.arange(1,41)*10) ## 40 different ESN size, The training should be increadibly quick
N_train=20000
N_checkpoint=19999
params = []
for T_data in T_datax:
    for N_ESN in N_ESNx:
        
        params.append([T_data,N_ESN])
import sys
print(len(params))
val_prc=0.1

for i in range(len(params)):
    param = params[i]
        
    T_data=param[0]
    N_ESN = param[1]
        


    Data=Data_Manager("MACKEY_manuscript",T_data)
    #N_ESN=500 ## Number of nodes in the ESN

    N_in=1          ## Input dimensionality
    N_av=10         ## Average number od connection for an ESN node
    alpha=0.9       ## Leakage term for the ESN node
    rho=0.99        ## Spectral radius
    gamma=0.15      ## Input scaling 
    T_seg=0
    Data.Init_ESN(N_ESN, N_in, N_av, alpha, rho, gamma)
    Data.ESN_Process()
    Data.Splitting(val_prc)

    F_Ns=torch.tensor([N_ESN,Data.Y_tr.size()[1]]) ## Definition of a simple read-out to be trained

    eta=0.0001 ## Learning rate
    weight_decay=0 ## Learning rate of the L2 penalty term
    model=Model_Learning(F_Ns, eta, weight_decay)

    T_seq=1
    
    N_train=20000
    N_checkpoint=19999
    
    N_check=int(np.ceil(N_train/N_checkpoint))
    ind_help=0

    N_chunks=1

    MSE_Train=torch.zeros([N_train])

    MSE_Tr=torch.zeros([N_check])
    MSE_Val=torch.zeros([N_check])
    MSE_Te=torch.zeros([N_check])

    t0s=torch.tensor(0.)
    batch_size=20

    for k in range(N_train):

        X_b, Y_b=Data.Batch(batch_size, T_seq)
        #print(X_b.shape,Y_b.shape)
        if model.N_ODE:

            var_help=X_b.clone()
            X_b=var_help[:,:,0:-1].clone()
            X_b=torch.concat([X_b,X_b],1)

            Y_b=var_help[:,:,1:]

        err,_=model.forward(X_b,Y_b,t0s)
        MSE_Train[k]=err.detach()

        if k%N_checkpoint==0: 


            if k>0:

                mse_mean=torch.mean(MSE_Train[k-N_checkpoint:k])

                MSE_Tr[ind_help]=mse_mean

            else:

                MSE_Tr[ind_help]=MSE_Train[k]


            ## VALIDATING ##
            #############

            X_val,Y_val=Data.Testing()

            if model.N_ODE:

                var_help=X_val.clone()
                X_val=var_help[:,:,0:-1].clone()
                X_val=torch.concat([X_val,X_val],1)

                Y_val=var_help[:,:,1:]

            err_, y_val=model.Seqforward(X_val,Y_val,t0s)

            MSE_Val[ind_help]=err_.detach()

            ## TESTING ##
            #############

            X_te,Y_te=Data.Testing(False)

            if model.N_ODE:

                var_help=X_te.clone()
                X_te=var_help[:,:,0:-1].clone()
                X_te=torch.concat([X_te,X_te],1)

                Y_te=var_help[:,:,1:]

            err_, y_te=model.Seqforward(X_te,Y_te,t0s)

            MSE_Te[ind_help]=err_.detach()

            Nx_plot=3
            Ny_plot=3

            fig, axs = plt.subplots(Nx_plot, Ny_plot,figsize=(20,10))

            print('Iteration number: ', k)
            for l in range(Nx_plot):

                for m in range(Ny_plot):

                    t_pred=l*Nx_plot+m
                    axs[l,m].plot(y_val.detach().to('cpu')[0,t_pred,:],'black')
                    #print(y_val.shape)
                    axs[l,m].plot(Y_val.detach().to('cpu')[0,t_pred,:],'red')

                    axs[l,m].legend(['Prediction','Target'])
                    #tit=
                    axs[l,m].title.set_text('Delay: '+str(t_pred))


            plt.savefig(os.path.join('ESN','plot_T_data_{}_N_ESN_{}.png'.format(T_data,N_ESN)))

            print('Errors: ', MSE_Tr[ind_help], MSE_Val[ind_help] , MSE_Te[ind_help])

            ind_help=ind_help+1

    torch.save(y_te,os.path.join('ESN','y_te_T_data_{}_N_ESN_{}.pt'.format(T_data,N_ESN)))
    torch.save(Y_te,os.path.join('ESN','xY_te_T_data_{}_N_ESN_{}.pt'.format(T_data,N_ESN)))
    torch.save(y_val,os.path.join('ESN','y_val_T_data_{}_N_ESN_{}.pt'.format(T_data,N_ESN)))
    torch.save(Y_val,os.path.join('ESN','xY_val_T_data_{}_N_ESN_{}.pt'.format(T_data,N_ESN)))
    torch.save(MSE_Tr,os.path.join('ESN','MSE_Tr_T_data_{}_N_ESN_{}.pt'.format(T_data,N_ESN)))
    torch.save(MSE_Val,os.path.join('ESN','MSE_Val_T_data_{}_N_ESN_{}.pt'.format(T_data,N_ESN)))
    torch.save(MSE_Te,os.path.join('ESN','MSE_Te_T_data_{}_N_ESN_{}.pt'.format(T_data,N_ESN)))


320
Iteration number:  0


: 